In [1]:
import pandas as pd
import numpy as np
import os.path

from keras.preprocessing.image import Iterator
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model

import threading
lock = threading.Lock()

from keras.callbacks import ModelCheckpoint

from BsonIterator import BSONIterator

Using TensorFlow backend.


In [2]:
METADATA_DIR = 'metadata'
DATA_DIR = '../../dataset'

train_offsets_df = pd.read_csv(os.path.join(METADATA_DIR,"train_offsets.csv"), index_col=0)
train_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Train_0.5_0.1.csv"), index_col=0)
val_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Val_0.5_0.1.csv"), index_col=0)

num_classes = 5270
num_train_images = len(train_images_df)
num_val_images = len(val_images_df)
batch_size = 64

# Create a generator for training and a generator for validation.
# Tip: use ImageDataGenerator for data augmentation and preprocessing.
assert os.path.exists(os.path.join(DATA_DIR, 'train.bson'))
train_bson_file = open(os.path.join(DATA_DIR, 'train.bson'), "rb")
train_datagen = ImageDataGenerator()
train_gen = BSONIterator(train_bson_file, train_images_df, train_offsets_df,
                         num_classes, train_datagen, lock, target_size = (224,224),
                         batch_size=batch_size, shuffle=True)

val_datagen = ImageDataGenerator()
val_gen = BSONIterator(train_bson_file, val_images_df, train_offsets_df,
                       num_classes, val_datagen, lock, target_size = (224,224),
                       batch_size=batch_size, shuffle=True)

/home/ankdesh/virtualenvs/cdiscount-py2.7/local/lib/python2.7/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Found 5570369 images belonging to 5270 classes.
Found 614245 images belonging to 5270 classes.


In [14]:
from keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.models import Model,load_model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications import mobilenet
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint

# Save model Dir
MODEL_DIR = 'saved_models'

# Checkpointer 
checkpointer = ModelCheckpoint(filepath=MODEL_DIR+'/Mobilenet_finetune.{epoch:02d}-{acc:.2f}-{val_acc:.2f}_RandomSplit_Train_0.5_0.1.h5', verbose=1)

# create the base pre-trained model
base_model = load_model(MODEL_DIR+'/Mobilenet.Epochs10_Acc0.58_Val0.52_RandomSplit_Train_0.9_0.1.h5', \
                  custom_objects={'relu6': mobilenet.relu6,'DepthwiseConv2D': mobilenet.DepthwiseConv2D})

model = Model(inputs=base_model.input, outputs=base_model.get_layer(u'conv_pw_13_relu').output).output
model = GlobalAveragePooling2D()(model)
model = Dense(1024, activation='relu')(model)
model = Dense(1024, activation='relu')(model)
model = Dense(5270, activation='softmax')(model)
model = Model(inputs=base_model.input, outputs=model)

In [23]:
for layer in model.layers[:46]:
    layer.trainable = False
for layer in model.layers[46:]:
    layer.trainable = True

In [30]:
# compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
opt = Adam(lr=0.00001)
model.compile(optimizer=opt,
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [31]:
num_epochs = 5


# train the model on the new data for a few epochs
model.fit_generator(train_gen,
                    steps_per_epoch = num_train_images // batch_size,
                    epochs = num_epochs,
                    validation_data = val_gen,
                    validation_steps = num_val_images // batch_size,
                    workers = 4, callbacks=[checkpointer])

Epoch 1/5
  199/87037 [..............................] - ETA: 20988s - loss: 4.8257 - acc: 0.2077

KeyboardInterrupt: 

In [11]:
for i,layer in enumerate(model.layers):
    print (i,layer.name)

(0, u'input_2')
(1, u'conv1')
(2, u'conv1_bn')
(3, u'conv1_relu')
(4, u'conv_dw_1')
(5, u'conv_dw_1_bn')
(6, u'conv_dw_1_relu')
(7, u'conv_pw_1')
(8, u'conv_pw_1_bn')
(9, u'conv_pw_1_relu')
(10, u'conv_dw_2')
(11, u'conv_dw_2_bn')
(12, u'conv_dw_2_relu')
(13, u'conv_pw_2')
(14, u'conv_pw_2_bn')
(15, u'conv_pw_2_relu')
(16, u'conv_dw_3')
(17, u'conv_dw_3_bn')
(18, u'conv_dw_3_relu')
(19, u'conv_pw_3')
(20, u'conv_pw_3_bn')
(21, u'conv_pw_3_relu')
(22, u'conv_dw_4')
(23, u'conv_dw_4_bn')
(24, u'conv_dw_4_relu')
(25, u'conv_pw_4')
(26, u'conv_pw_4_bn')
(27, u'conv_pw_4_relu')
(28, u'conv_dw_5')
(29, u'conv_dw_5_bn')
(30, u'conv_dw_5_relu')
(31, u'conv_pw_5')
(32, u'conv_pw_5_bn')
(33, u'conv_pw_5_relu')
(34, u'conv_dw_6')
(35, u'conv_dw_6_bn')
(36, u'conv_dw_6_relu')
(37, u'conv_pw_6')
(38, u'conv_pw_6_bn')
(39, u'conv_pw_6_relu')
(40, u'conv_dw_7')
(41, u'conv_dw_7_bn')
(42, u'conv_dw_7_relu')
(43, u'conv_pw_7')
(44, u'conv_pw_7_bn')
(45, u'conv_pw_7_relu')
(46, u'conv_dw_8')
(47, u'conv

In [19]:
model.save('saved_models/Mobilenet_finetune.2denselayer.01-0.44-0.45_RandomSplit_Train_0.5_0.1.h5')